In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# LAB MODULE 1.
# Introduction to geospatial data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Topics covered:
# (1) Vector and raster data types
# (2) Geographic and projected coordinate systems
# (3) Basic plots 


# Links showing examples for plotting and calculating distances using different methods
#  https://stackoverflow.com/questions/57291951/how-to-calculate-geodesic-distance-along-a-path-lat-lon-points-at-once
#  https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac
#  https://www.earthdatascience.org/
#  http://www.naturalearthdata.com/

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Exercise 1.
# Projection systems and shape of the Earth
#
# https://scitools.org.uk/cartopy/docs/latest/index.html
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
#import cartopy.feature as cfeature

In [ ]:
# Example 1
#
# Store location names and geographic coordinates
# Example: Milano - geographical coordinates from Wikipedia
# sessagesimali 	45° 28′ 1″ N, 9° 11′ 24″ E
# Decimali 	45.466944°, 9.19°
# nome città ; latitudine (gradi N) ; longitudine (gradi E)

cities = ('Madrid \n(Spain)',\
          'Beijing \n(China)',\
          "Abidjan \n(Cote d'Ivoire)",\
          'Bandar Seri Begawan \n(Brunei)')
lats = (40.50,\
          39.91,\
          5.34,\
          4.97 )
lons = (-3.67,\
          116.39,\
          -4.03,\
          114.97 )
print (cities[2])

In [ ]:
# Define the coordinate system in which the coordinates of your data are given
#   https://www.esri.com/arcgis-blog/products/arcgis-pro/mapping/gcs_vs_pcs/
data_crs = ccrs.Geodetic()         # geographical
#data_crs = ccrs.PlateCarree()      # projected


# Set a map with a given projection for visualizatin (then play with it)
myproj = ccrs.PlateCarree()
#myproj = ccrs.Mercator()
#myproj = ccrs.Mollweide()
#myproj = ccrs.AlbersEqualArea()
#myproj = ccrs.NearsidePerspective(central_longitude=50.0, central_latitude=10.0)
ax = plt.axes(projection=myproj)


# Add some features (you can play a bit)
ax.coastlines()
#ax.gridlines(draw_labels=True)
ax.stock_img()

# Add locations on the map
for i in range(0,4):
  plt.plot(lons[i],lats[i],color='r',marker='o',transform=data_crs)

# Add names on the map
for i in range(0,4):
  plt.text(lons[i]-50,lats[i]+5,cities[i],color='cyan',transform=data_crs)


# Plot the distance between couples of cities (shortest or along the same latitude, depending on transform)
plt.plot([lons[0],lons[1]],[lats[0],lats[1]],color='blue',transform=data_crs) # Madrid-Beijing
plt.plot([lons[2],lons[3]],[lats[2],lats[3]],color='green',transform=data_crs) # Abidjan-Brunei
plt.plot([lons[0],lons[2]],[lats[0],lats[2]],color='orange',transform=data_crs) # Madrid-Abidjan
plt.plot([lons[1],lons[3]],[lats[1],lats[3]],color='brown',transform=data_crs) # Beijing-Brunei


# Save the plot by calling plt.savefig() BEFORE plt.show()
plt.savefig('Module1_plot02.pdf')
plt.show()

In [ ]:
# Example 2
#
# Now let's try to plot raster data
#  Import data from a table
#  You see it's organized as a 2D table, with first row and column containing the coordinates
#  The elements in the 2D table represent the variable to plot
#  The first row taken as column name ny default; we specify to take first coulumn as names, too.
#  Then we also extract longitudes and latitudes

# Think of other ways this kind of data could be organized and imported into python data structures

import pandas as pd

raster_data = pd.read_csv('./Module1_gridded_data.txt',sep='\t',index_col=0)

In [ ]:
print(raster_data.head())

In [ ]:
# Get coordinates

lons = pd.read_csv('./Module1_gridded_data.txt',sep='\t',nrows=1,header=None)
print(lons[0])
lons.drop([0], axis=1, inplace=True)
lons = lons.transpose()
print(lons)

lats = pd.read_csv('./Module1_gridded_data.txt',sep='\t',usecols=[0])
print(lats)

In [ ]:
# Set a global map with chosen projection and plot the raster data

import matplotlib as mpl

fig = plt.figure(figsize=(9,6))  # x,y(inches)

ax = plt.axes(projection=ccrs.Robinson())
ax.set_global()
ax.set_title('Unknown variable')

mm = ax.pcolormesh(lons.transpose(), lats, raster_data, \
                   transform=ccrs.PlateCarree(),cmap=mpl.cm.cubehelix )

# more color palettes here:
#   https://matplotlib.org/3.1.3/tutorials/colors/colormaps.html

ax.coastlines()

#- add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('unknown units') #($\mu g$ $m^{-3}$)
cbar.ax.tick_params(labelsize=8)

plt.savefig('Module1_plot03.pdf')
plt.show()
plt.close()

In [ ]:
# Example 2 
# Use the geographic grid covering the whole surface of the globe, 
#   and highlight with a color scale the approximate areas of different grid cells.

# Hints: - assume that the Earth is a sphere of radius 6371 km
#        - each grid cell is approximately a trapezoid

In [ ]:
# There are several possible strategies, for instance:
#  1- use geometric calculations relative to the sphere
#  2- divide the sphere into n trapezoids
#  3- use geopandas geometry to calculate areas, but first you need to convert the grid into a series of polygons

In [ ]:
# I can calculate the surface of the Earth, and use it later to double-check my calculations
#   https://en.wikipedia.org/wiki/Sphere
er = 6371
es = 4*3.141618*(er**2)
print(es)

In [ ]:
# In a regular latlon grid
# the area of grid cells is the same for all grid cells in the same latitude band, independent of the longitude 

In [ ]:
# Solution (1)

# - Divide the world into latitude bands 
#   (in our case the grid spacing is 10x10, so 18 latitude bands cover -90+90 deg.)
# - Calculate the area of each latitude band, using the formula for spherical caps
#   https://en.wikipedia.org/wiki/Spherical_cap
#   https://www.pmel.noaa.gov/maillists/tmap/ferret_users/fu_2004/msg00023.html
# - Divide the area of each latitude band by the number of longitude bands
#   (36 longitude bands cover -180+180 deg.E)

In [ ]:
# Let's start from the "top" spherical cap, that around the North Pole until 80N
#   The area north of a line of latitude is: A = 2*pi*R^2(1-sin(lat)), with lat in radians

from math import pi,sin

A8090Na = 2 * pi * (er**2) * (1-sin(80*pi/180))

# let's also substitute 80 with values of lats form our grid, which are identified by the center rather than margins
A8090Nb = 2 * pi * (er**2) * (1-sin((lats.iloc[0]+lats.iloc[1])/2*pi/180))
print(A8090Na,A8090Nb)

In [ ]:
# Now extend this case
# The area between two lines of latitude (latitude band) is the difference between the area north of latitude A 
#     and the area north of latitude B.
# A = 2*pi*R^2 |sin(lat1)-sin(lat2)| |lon1-lon2|/360

# Let's first create a more convenient array to contain lower latitude margins of the grid cell
#   expressed in radians, then solve with a loop

import numpy as np

latsr = np.empty([len(lats)],dtype=float)
area = np.empty([len(lats),len(lons)],dtype=float)

for j in range(0,len(lats)):
  latsr[j]=(lats.iloc[j]-5)*pi/180
  print(j,lats.iloc[j],latsr[j],'\n')

# Our last latitude band should be 80S-90S, and identified by 90S, i.e. -π/2; let's check
print(latsr[-1:],-pi/2)

In [ ]:
for j in range(1,len(lats)-1):
  area[j,:]= 2*pi*(er**2) * abs(sin(latsr[j])-sin(latsr[j-1])) * 10/360
  print(j,lats.iloc[j],lats.iloc[j]-5,lats.iloc[j-1]-5,\
        latsr[j],latsr[j+1],area[j,0],'\n')
area[0,:]= A8090Nb * 10/360
area[-1,:]= area[0,:]

area=area/1e3 # express it in thousands of square kilometers
print(area[:,4])

In [ ]:
fig = plt.figure(figsize=(9,6))  # x,y(inches)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.set_title('Grid cell area (10 x 10 deg.)\n')

mm = ax.pcolormesh(lons.transpose(), lats, area, \
                   transform=ccrs.PlateCarree(),cmap=mpl.cm.cubehelix )

# more color palettes here:
#   https://matplotlib.org/3.1.3/tutorials/colors/colormaps.html

ax.coastlines()
ax.gridlines(draw_labels=True)

#- add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('$thousand$ $km^{2}$')
cbar.ax.tick_params(labelsize=8)

plt.savefig('Module1_plot04.pdf')
plt.show()
plt.close()

In [ ]:
# Let's double-check: 
# - the areas for latitude bands should be symmetrical about the Equator
# - the sum of areas of all grid cells should sum up to the area of the sphere calculated initially

In [ ]:
# The symmetry is there in the data, it's displaced by effect of plotting!
#  This "issue" is standing out very clearly in our example
#  https://bairdlangenbrunner.github.io/python-for-climate-scientists/matplotlib/pcolormesh-grid-fix.html

In [ ]:
# Let's extend our "latsr" and use it instead

In [ ]:
#area_glob = np.empty([1],dtype=float)       
area_glob = np.sum(area[:,:])*1000.
print(area_glob)

In [ ]:
print(area_glob/es)